In [1]:
# Gaussian Naive Bayes

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools

# 1 -- Load Data

In [3]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [4]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [5]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [6]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [7]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [8]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [9]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [10]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_game_correct
ros_game_incorrect
ros_game_start
ros_games_session
ros_in_game
ros_mistake_made
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [11]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [12]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

### Filter Feature Set (Optional)

In [13]:
# features_to_keep = data[0].columns

# always include basic_cols, add desired group of features 
features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [14]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [15]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [16]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for training
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i])
    
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i], split_size)
            train_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i])
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)

    y_train = train_data['engagement']
    X_train = train_data.drop(columns=basic_cols)
    y_test = test_data['engagement']
    X_test = test_data.drop(columns=basic_cols)        

    # Sort Training Data Together 
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        X_train[c] = (X_train[c]-mean)/(std)
        X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [17]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.naive_bayes import GaussianNB

# Hyperparameter Combinations to Try:  
hp_names = ['var_smoothing']
hp_list = [[1e-12,1e-9,1e-8,1e-6]]

def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    clf = GaussianNB(var_smoothing=hp['var_smoothing'])
    clf.fit(X_train.values, y_train.values)

    pred = clf.predict(X_test.values)
    scores = clf.predict_proba(X_test.values)[:,1]
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [18]:
all_results = []
for n in range(1, len(part)):
    comb = combinations(part, n)
    print()
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(train_part, test_part, 0)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
        # Set Up Cross Validation Groups
        div = int(len(X_train)/3)
        X_cv1 = X_train.loc[:div]
        X_cv2 = X_train.loc[div+1:div*2]
        X_cv3 = X_train.loc[(div*2)+1:]
        
        y_cv1 = y_train.loc[:div]
        y_cv2 = y_train.loc[div+1:div*2]
        y_cv3 = y_train.loc[(div*2)+1:]
        
        cv1_X_train = X_cv1.append(X_cv2)
        cv1_X_test = X_cv3
        cv1_y_train = y_cv1.append(y_cv2)
        cv1_y_test = y_cv3
        
        cv2_X_train = X_cv1.append(X_cv3)
        cv2_X_test = X_cv2
        cv2_y_train = y_cv1.append(y_cv3)
        cv2_y_test = y_cv2

        cv3_X_train = X_cv3.append(X_cv2)
        cv3_X_test = X_cv1
        cv3_y_train = y_cv3.append(y_cv2)
        cv3_y_test = y_cv1

        # Hyperparameter Combinations        
        hp_combos = list(itertools.product(*hp_list))
        best, best_acc = None, 0
        for h in hp_combos:
            hp = {}
            for i,v in enumerate(h):
                hp[hp_names[i]] = v
            
            # Cross-Validation
            res1 = model(cv1_X_train, cv1_y_train, cv1_X_test, cv1_y_test, hp, False)
            res2 = model(cv2_X_train, cv2_y_train, cv2_X_test, cv2_y_test, hp, False)
            res3 = model(cv3_X_train, cv3_y_train, cv3_X_test, cv3_y_test, hp, False)
            
            acc = np.mean([res1['accuracy'], res2['accuracy'], res3['accuracy']])
            print(hp, acc)
            if acc > best_acc:
                best = hp
                best_acc = acc
        
        # Evaluate on Test Using Best Hyperaparemeters  
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
{'var_smoothing': 1e-12} 0.3498952504670105
{'var_smoothing': 1e-09} 0.41299244341938035
{'var_smoothing': 1e-08} 0.4396666502057629
{'var_smoothing': 1e-06} 0.5220093481234593
Best HP {'var_smoothing': 1e-06}
{'f1_1': 0.5858388760689561, 'recall_1': 0.4844942748091603, 'auc': 0.6824448018103377, 'recall_0': 0.7089546845262473, 'precision_0': 0.4447668707020328, 'f1_0': 0.5466129241971135, 'accuracy': 0.5671126954146826, 'precision_1': 0.7407955715756952}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
{'var_smoothing': 1e-12} 0.7417762295885053
{'var_smoothing': 1e-09} 0.7433165677020686
{'var_smoothing': 1e-08} 0.7481859008652244
{'var_smoothing': 1e-06} 0.761005332299706
Best HP {'var_smoothing': 1e-06}
{'f1_1': 0.8051610495378115, 'recall_1': 0.8613923509112169, 'auc': 0.6610313987908661, 'recall_0': 0.44956845489930614, 'precision_0': 0.6211855489302, 'f1_0': 0.521623877080163, 'accuracy': 0.7231009660920629, 'precision_1': 0.7558213736064363}


/anaconda3/envs/caispp/lib/python3.5/site-packages/ipykernel/__main__.py:49: RuntimeWarning: All-NaN axis encountered


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [24]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [25]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"7, 9, 11, 12, 17, 18",5
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"5, 9, 11, 12, 17, 18",7
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"5, 7, 11, 12, 17, 18",9
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"5, 7, 9, 12, 17, 18",11
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"5, 7, 9, 11, 17, 18",12


In [ ]:
results.to_csv('results.csv', index=False)

In [23]:
for d in data:
    for c in d.columns:
        if d[c].isna().sum() == len(d):
            print(c)